In [ ]:
import openai

client = openai.OpenAI()

PROMPT = """
I have the following function in my system.

`get_weather`
`get_currency`
`get_news`

All of the receive the name of a country as an argument (i.e get_news('Spain'))

Please answer with the name of the function that you would like me to run.

Please say nothing else, just the name of the function with the arguments.

Anser the following question:

What is the weather in Greece?

"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": PROMPT
        }
    ]
)

response

ChatCompletion(id='chatcmpl-CIOsRdkdq2ApUajqSexb4ZyDv6GpA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of Greece is Athens.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1758501567, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=7, prompt_tokens=14, total_tokens=21, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [8]:
message = response.choices[0].message.content

message

'The capital of Greece is Athens.'